In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import openpyexcel
from openpyexcel import workbook, load_workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyexcel.utils import get_column_letter
from itertools import islice
from os import listdir
import re
import csv
import datetime
import pickle

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:.0f}'.format

In [3]:
company = pd.read_csv('CSV/company_ws.csv',index_col=0)
company

,Balance,Income,Cash
ADBE1.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE2.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE3.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE4.xlsx,Consolidated Balance Sheets,Consolidated Statements of Inco,Consolidated Statements of Cash
ADBE5.xlsx,Consolidated_Balance_Sheets,Consolidated_Statements_of_Inc,Consolidated_Statements_of_Cas
AME1.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME2.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME3.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME4.xlsx,Consolidated Balance Sheet,Consolidated Statement of Incom,Consolidated Statement of Cash
AME5.xlsx,Consolidated_Balance_Sheet,Consolidated_Statement_of_Inco,Consolidated_Statement_of_Cash


In [30]:
def clean_table(table):
        
    df = table.copy()
    df.replace(u'\xa0','', regex=True, inplace=True)
    
    # Replace empty string with nan
    df.replace("^$",np.nan, regex=True, inplace=True)
    
    # Replace fields with [n] as nan
            
    for i in range(1,len(df.columns)):
      if df[df.columns[i]].dtype == object:
         df[df.columns[i]].replace("\[\d\]*$", np.nan, regex=True, inplace=True)
    
    # Remove footnotes embedded in table
    
    df.drop(df.loc[df[df.columns[0]].str.contains(r"^\[\d?\]$", case=False, regex=True,na=False)].index,inplace=True)
    
    # Replace nan in first column with ''
    
    df[df.columns[0]].replace(np.nan,'',inplace=True)
    
    #Remove blank columns and rows
    
    df.dropna(axis=1, how='all', inplace=True)
    df.dropna(axis=0, how='all', inplace=True)
    
    if len(df.columns) > 5:
      # Take the first and the last 3 columns
        
      a = df.iloc[:,:1].copy()
      b = df.iloc[:,-3:].copy()
      df = pd.concat([a,b], axis=1).copy()
    
    #Remove comments in brackets eg (Notes 1) and spurious characters
    
    df[df.columns[0]].replace(r"[\s]?\(Note.*\)", "", regex=True, inplace=True)
    df[df.columns[0]].replace(r"â€™","", regex=True, inplace=True)
    
    # Deal with situation where column labels are of type date time. Extract year and convert to str
    
    col = list(df.columns[0:])
 
    for i in range(0,len(col)):
      if type(col[i]) is datetime.datetime :
        col[i] = col[i].strftime('%Y')
    
    # Set columns to extraxted year which will be of type string
    # If we don't do this, the following if condition will not work.
    
    df.columns = col
    
    if df.columns.str.contains(r"\d\d\d\d", case=False, regex=True,na=False).sum() == 0:
        # Take Column Names from first row
        
        df.iloc[0].fillna('', axis=0, inplace=True)
        
        # Deal with the situation where the first row is read in as date time
    
        for i in range(0,len(df.iloc[0])):
          if type(df.iloc[0][i]) is datetime.datetime :
            df.iloc[0][i] = df.iloc[0][i].strftime('%Y')
            x=1
            
        df.iloc[0] = df.iloc[0].astype('string')
        col = list(df.iloc[0])
        
        # Drop the first row
    
        df.drop(index=df.index[0], axis=0, inplace=True)
        
    col[0] = 'Index'
    #print(col) 
    
    # Extraxt year if there is a match
    
    for i in range(1,len(col)):
      res = re.search('(20[0-9][0-9])', col[i])

      if res:
        col[i] = res.group(0)
    
    print(col) 
    df.columns = col  
    
    # Remove footnotes embedded in table
    
    df.drop(df.loc[df[df.columns[0]].str.contains(r"^\[\d?\]$", case=False, regex=True,na=False)].index,inplace=True)
    
    # Drop columns and headers that are blank
    
    #df.dropna(axis=1, how='all', inplace=True)
    #df.dropna(axis=0, how='all', inplace=True)
    
    # Make sure that first column doesn't have any nulls
  
    df['Index'] = df['Index'].fillna('')
    
    # Set index to first column
    
    df.set_index(df.columns[0], inplace=True)
    
    #Replace single quote with nan
    
    df.replace("'",np.nan, inplace=True)
    
    # Remove rows which contain otherwise specified
    
    df = df.loc[~df.index.str.contains(rf"(?:Consolidated|Income|unless otherwise)",case=False, regex=True,na=False)]
    
    # Show first row as a validation check
    
    print(pd.DataFrame(df.iloc[0]).T)
    
    return df

In [5]:
def get_line_items(t):
 
  
  heading = []
  line_item = []
    
  for index, row in t.iterrows():  
    
    if type(index) is tuple:
        print("<<Tuple Found>>")
        
    if row.replace('', np.nan).isna().all():
      # Blank row, so assume heading
    
      heading.append(index)
    else:    
      line_item.append(index)


  return {'heading' : heading,
          'line_item' : line_item}

In [6]:
def get_spreadsheet(company,ticker,file, type):
    
  rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

  dirname = rootdir + ticker
  path = dirname + '/' + file

  xl_file = pd.ExcelFile(path)
  table = xl_file.parse(company.loc[file,type],parse_dates=False)

  return table

In [32]:
rootdir = "C:/Users/priye/Desktop/Dissertation/DATA/"

income_sheets = {}
headings = []
line_items = []

for d in listdir(rootdir):

  sheets = []
  dirname = rootdir + d
 
  for f in listdir(dirname):
    if f.endswith('xlsx'):
      print(f)
      table = get_spreadsheet(company,d,f,'Income')
      t = clean_table(table)
      sheets.append(t)
        
      l_dict = get_line_items(t)      
      headings = headings + l_dict['heading']
      line_items = line_items + l_dict['line_item']
  income_sheets[d] = sheets
    
# Remove duplicates from headings and line items

unique_list = set(headings)
headings = list(unique_list)

unique_list = set(line_items)
line_items = list(unique_list)

table_labels = {'headings' : headings,
                'line_items' : line_items}


ADBE1.xlsx
['Index', '2022', '2021', '2020']
         2022 2021 2020
Revenue:  NaN  NaN  NaN
ADBE2.xlsx
['Index', '2020', '2019', '2018']
         2020 2019 2018
Revenue:  NaN  NaN  NaN
ADBE3.xlsx
['Index', '2018', '2017', '2016']
         2018 2017 2016
Revenue:  NaN  NaN  NaN
ADBE4.xlsx
['Index', '2016', '2015', '2014']
         2016 2015 2014
Revenue:  NaN  NaN  NaN
ADBE5.xlsx
['Index', '2014', '2013', '2012']
         2014 2013 2012
Revenue:  NaN  NaN  NaN
ADM1.xlsx
['Index', '2022', '2021', '2020']
            2022   2021   2020
Revenues  101556  85249  64355
ADM2.xlsx
['Index', '2020', '2019', '2018']
           2020   2019   2018
Revenues  64355  64656  64341
ADM3.xlsx
['Index', '2018', '2017', '2016']
           2018   2017   2016
Revenues  64341  60828  62346
ADM4.xlsx
['Index', '2018', '2017', '2016']
           2018   2017   2016
Revenues  64341  60828  62346
ADM5.xlsx
['Index', '2014', '2013', '2012']
           2014   2013   2012
Revenues  81201  89804  89038
AME1.xlsx
['I

['Index', '2020', '2019', '2018']
             2020 2019 2018
Net revenues  NaN  NaN  NaN
FFIV3.xlsx
['Index', '2018', '2017', '2016']
             2018 2017 2016
Net revenues  NaN  NaN  NaN
FFIV4.xlsx
['Index', '2016', '2015', '2014']
             2016 2015 2014
Net revenues  NaN  NaN  NaN
FFIV5.xlsx
['Index', '2014', '2013', '2012']
              2014  2013  2012
Net revenues   NaN   NaN   NaN
GILD1.xlsx
['Index', '2022', '2021', '2020']
          2022 2021 2020
Revenues:  NaN  NaN  NaN
GILD2.xlsx
['Index', '2020', '2019', '2018']
          2020 2019 2018
Revenues:  NaN  NaN  NaN
GILD3.xlsx
['Index', '2018', '2017', '2016']
         2018 2017 2016
Revenues  NaN  NaN  NaN
GILD4.xlsx
['Index', '2016', '2015', '2014']
         2016 2015 2014
Revenues  NaN  NaN  NaN
GILD5.xlsx
['Index', '2014', '2013', '2012']
                    2014 2013 2012
Revenues [Abstract]  NaN  NaN  NaN
GM1.xlsx
['Index', '2022', '2021', '2020']
                      2022 2021 2020
Net sales and revenue  NaN  Na

['Index', '2016', '2015', '2014']
                    2016   2015   2014
Operating Revenues  9061  10415  10886
PEG5.xlsx
['Index', '2014', '2013', '2012']
                     2014  2013  2012
Operating Revenues  10886  9968  9781
Pepsico1.xlsx
['Index', '2022', '2021', '2020']
              2022   2021   2020
Net Revenue  86392  79474  70372
Pepsico2.xlsx
['Index', '2019', '2018', '2017']
              2019   2018   2017
Net Revenue  67161  64661  63525
Pepsico3.xlsx
['Index', '2015', '2014', '2013']
              2015   2014   2013
Net Revenue  63056  66683  66415
REGN1.xlsx
['Index', '2022', '2021', '2020']
          2022 2021 2020
Revenues:  NaN  NaN  NaN
REGN2.xlsx
['Index', '2020', '2019', '2018']
          2020 2019 2018
Revenues:  NaN  NaN  NaN
REGN3.xlsx
['Index', '2018', '2017', '2016']
          2018 2017 2016
Revenues:  NaN  NaN  NaN
REGN4.xlsx
['Index', '2016', '2015', '2014']
          2016 2015 2014
Revenues:  NaN  NaN  NaN
REGN5.xlsx
['Index', '2014', '2013', '2012']
 

In [19]:
table = get_spreadsheet(company,'CSCO','CSCO5.xlsx','Income')
df = table.copy()
df

,Consolidated Statements Of Operations (USD $),12 Months Ended,Unnamed: 2,Unnamed: 3
0,"In Millions, except Per Share data, unless otherwise specified","Jul. 26, 2014","Jul. 27, 2013","Jul. 28, 2012"
1,REVENUE:,',','
2,Product,36172,38029,36326
3,Service,10970,10578,9735
4,Total revenue,47142,48607,46061
5,COST OF SALES:,',','
6,Product,15641,15541,14505
7,Service,3732,3626,3347
8,Total cost of sales,19373,19167,17852
9,GROSS MARGIN,27769,29440,28209


In [20]:
k = clean_table(df)
k

['Index', '2014', '2013', '2012']
         2014 2013 2012
REVENUE:    '    '    '


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:109: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


,2014,2013,2012
Index,,,
REVENUE:,',','
Product,36172,38029,36326
Service,10970,10578,9735
Total revenue,47142,48607,46061
COST OF SALES:,',','
Product,15641,15541,14505
Service,3732,3626,3347
Total cost of sales,19373,19167,17852
GROSS MARGIN,27769,29440,28209


In [15]:
def save_line_items(table_labels):
    
  with open('CSV/income_headings.csv', 'w',encoding='utf-8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(table_labels['headings'])
    
  with open('CSV/income_line_items.csv', 'w',encoding='utf-8') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(table_labels['line_items'])

In [16]:
def save_table_labels(table_labels):

  with open('pickle/income_table_labels.pkl', 'wb') as fp:
    pickle.dump(table_labels, fp)


In [17]:
def save_income_sheets(income_sheets):
  with open('pickle/income_sheets.pkl', 'wb') as fp:
    pickle.dump(income_sheets, fp)

In [117]:
save_table_labels(table_labels)

In [118]:
save_income_sheets(income_sheets)

In [18]:
income_sheets['CAT'][1]

,2020,2019,2018
Index,,,
Sales and revenues:,NaN,NaN,NaN
Sales and revenues,41748,53800,54722
Operating costs:,NaN,NaN,NaN
Cost of goods sold,29082,36630,36997
"Selling, general and administrative expenses",4642,5162,5478
Research and development expenses,1415,1693,1850
Total operating costs,37195,45510,46429
Operating profit,4553,8290,8293
Profit (loss),2998,6093,6147
